### Creación de Tablas Auxiliares para el Análisis

Con el objetivo de enriquecer el modelo de datos y facilitar el análisis posterior, se procede a la creación de una serie de **tablas auxiliares** derivadas de los datos existentes. Estas tablas permiten estructurar relaciones implícitas y desnormalizar información contenida en campos anidados o de tipo `JSONB`.

#### Objetivos de esta fase:
- Representar relaciones **muchos-a-muchos**, como productos por transacción.
- Descomponer y estructurar información contenida en campos `JSONB` como `product_metadata` o `event_metadata`.
- Mejorar la accesibilidad de los datos y optimizar consultas analíticas posteriores.

Estas tablas auxiliares no forman parte del modelo relacional inicial, pero se consideran fundamentales para los análisis de comportamiento, segmentación y personalización del sistema de recomendación.


## Creación de Tablas Auxiliares directamente en la Base de Datos

Durante el desarrollo del sistema de análisis de comportamiento y recomendación, se ha optado por la creación de **tablas auxiliares directamente en la base de datos (PostgreSQL)** en lugar de procesarlas con dataframes.

### Ventajas de esta aproximación

#### 1. Eficiencia en el procesamiento
PostgreSQL está optimizado para trabajar con grandes volúmenes de datos, permitiendo operaciones como desanidado de JSONB, agregaciones o joins complejos de forma mucho más rápida que si los datos se cargan y manipulan en memoria desde Python o herramientas de BI.

#### 2. Persistencia y reutilización
Al crear las tablas auxiliares en la base de datos, los datos derivados quedan almacenados de forma persistente y pueden ser reutilizados por múltiples procesos:
- Dashboards en Power BI,
- Scripts de análisis en Python,
- Algoritmos de machine learning,
- Exportaciones para reporting.

#### 3. Optimización de consultas analíticas
Estas tablas desnormalizadas y estructuradas facilitan consultas analíticas complejas, reduciendo el tiempo de cómputo y permitiendo una mejor planificación de índices, claves y relaciones entre entidades.

#### 4. Integridad relacional
Definir claves foráneas, restricciones y relaciones directamente en SQL permite mantener la **integridad del modelo de datos**, lo que se traduce en mayor fiabilidad en los análisis y menor probabilidad de errores.

#### 5. Automatización del flujo ETL
Incorporar estas operaciones al sistema de base de datos facilita la automatización y escalabilidad del flujo de procesamiento:
- Se puede ejecutar mediante scripts programados (cron jobs, Airflow, etc.),
- O integrarse en pipelines de datos empresariales.

---

### Conclusión

Crear las tablas auxiliares directamente en la base de datos proporciona una solución **más robusta, escalable y eficiente**, alineada con las necesidades analíticas del proyecto. Este enfoque también favorece la trazabilidad y reproducibilidad de los análisis, aspectos clave en entornos profesionales de datos.


In [1]:
from sqlalchemy import create_engine,text
import pandas as pd
import psycopg2
from dotenv import load_dotenv
import os
import json

### Conexión a la BBDD

In [2]:
# Load .env file
load_dotenv(override=True)

# Configuración de conexión
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_PORT = os.getenv("DB_PORT", "5432")  # Default port

# Crear la conexión con SQLAlchemy
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")


# Creación de tabla de **'products_transaction'**

### Creación de la Tabla Auxiliar `Products_Transactions`

Con el objetivo de representar de forma explícita la relación muchos-a-muchos entre productos y transacciones, se ha creado una tabla auxiliar denominada `Transaction_Products`. Esta tabla permite descomponer la información contenida en el campo `product_metadata` de la tabla `Transactions`, el cual almacena un array JSON con los productos comprados en cada sesión.

#### Extracción desde JSONB
Para ello, se utilizó la función `jsonb_array_elements()` de PostgreSQL, que permite expandir arrays de tipo `JSONB` almacenados en una columna. La consulta SQL generó una fila por cada producto incluido en el campo `product_metadata`, extrayendo los siguientes atributos:
- `session_id`: identificador de la transacción,
- `product_id`: identificador del producto,
- `quantity`: cantidad comprada,
- `item_price`: precio unitario del producto.

#### Ventajas de esta aproximación
- Mejora la normalización del modelo de datos.
- Permite realizar análisis más detallados por producto, como frecuencia de compra o valor total generado.
- Facilita la integración con la tabla `Products` mediante `JOIN` por `product_id`.

#### Estructura de la nueva tabla
La información extraída puede ser almacenada en una tabla llamada `Transaction_Products`, con la siguiente estructura:

```sql
CREATE TABLE IF NOT EXISTS Products_Transactions (
    session_id UUID REFERENCES Transactions(session_id),
    product_id INT REFERENCES Products(id),
    quantity INT,
    item_price FLOAT,
    PRIMARY KEY (session_id, product_id)
);
```
Esta tabla servirá como base para futuras métricas como el valor total por transacción, productos más vendidos, análisis RFM y segmentación basada en comportamiento de compra.

In [3]:
%%time
# Crear conexión SQLAlchemy
DATABASE_URL = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)

# Script SQL: crear tabla + insertar datos
sql_script = """
-- Crear tabla si no existe
CREATE TABLE IF NOT EXISTS Products_Transactions (
    session_id UUID REFERENCES Transactions(session_id),
    product_id INT REFERENCES Products(id),
    quantity INT,
    item_price FLOAT
);

-- Insertar datos desanidados desde product_metadata
INSERT INTO Products_Transactions (session_id, product_id, quantity, item_price)
SELECT 
    t.session_id,
    (json_data->>'product_id')::INT       AS product_id,
    (json_data->>'quantity')::INT         AS quantity,
    (json_data->>'item_price')::FLOAT     AS item_price
FROM (
    SELECT session_id, jsonb_array_elements(product_metadata) AS json_data
    FROM Transactions
) AS t;
"""

# Ejecutar el script
with engine.connect() as conn:
    try:
        with engine.begin() as conn:  # begin() gestiona transacciones automáticamente
            conn.execute(text(sql_script))
        print("✅ Tabla 'Products_Transactions' creada y poblada con éxito.")
    except Exception as e:
        print("❌ Error al ejecutar el script:", e)


✅ Tabla 'Products_Transactions' creada y poblada con éxito.
CPU times: total: 46.9 ms
Wall time: 21.7 s


### Creación de Tablas Auxiliares a partir de `event_metadata`

Con el objetivo de normalizar y estructurar la información contenida en el campo `event_metadata` de la tabla `Click_Stream`, se procede a la creación de una serie de tablas auxiliares que descomponen dicho campo según el tipo de evento.

#### Motivación:
- El campo `event_metadata` almacena información adicional en formato `JSONB`, que varía según el `event_name`.
- Descomponer esta información permite acceder a sus atributos de forma directa, mejorar la capacidad de consulta y construir indicadores personalizados.

#### Proceso general:
1. Se analiza la estructura del campo `event_metadata` para cada tipo de evento.
2. Se definen tablas auxiliares específicas con los atributos relevantes.
3. Se mantiene `event_id` como clave foránea, permitiendo enlazar con el evento original.

#### Objetivos:
- Mejorar la accesibilidad y trazabilidad de los datos.
- Permitir análisis más precisos por tipo de interacción.
- Facilitar la construcción de visualizaciones y modelos basados en comportamiento detallado.

> Esta estrategia de descomposición transforma los datos semi-estructurados en estructuras relacionales, alineadas con las buenas prácticas de modelado analítico.


### Exploración del Campo `event_metadata`

Antes de proceder a la creación de tablas auxiliares, se realizó una exploración del contenido del campo `event_metadata` de la tabla `Click_Stream`, con el objetivo de identificar las estructuras y claves más comunes asociadas a cada tipo de evento.

#### Procedimiento:
- Se extrajo una muestra representativa de 1000 registros con `event_metadata` no nulo.
- Se agruparon los resultados por tipo de evento (`event_name`) para observar las diferencias y similitudes en la estructura de los metadatos.
- Se amplió el ancho de visualización en el entorno de Jupyter para facilitar la lectura del contenido JSON.

#### Objetivo:
- Detectar patrones y campos recurrentes dentro de los metadatos.
- Determinar qué atributos deben ser extraídos a tablas auxiliares específicas.
- Sentar las bases para un modelo de datos relacional más rico y accesible.

> Esta fase exploratoria es clave para transformar datos semi-estructurados en estructuras tabulares bien definidas, manteniendo la flexibilidad original del evento pero facilitando su explotación analítica.


In [4]:
# Query: muestra aleatoria de eventos con metadatos
query = """
SELECT event_name, event_metadata
FROM click_stream
WHERE event_metadata <>'{}'
LIMIT 1000;
"""

# Leer los datos
df_sample = pd.read_sql(query, engine)

# Ampliar visibilidad en Jupyter
pd.set_option('max_colwidth', 500)

# Mostrar primeras filas
display(df_sample.head())

# Agrupar por event_name y tomar los primeros 3 ejemplos por grupo
sample_by_event = df_sample.groupby("event_name").apply(lambda x: x.head(3))[["event_metadata"]]

# Resetear el índice para mostrarlo bien
sample_by_event = sample_by_event.reset_index(drop=True)

# Mostrar resultado
display(sample_by_event)

,event_name,event_metadata
0,ADD_TO_CART,"{'quantity': 4, 'item_price': 313529, 'product_id': 15315}"
1,BOOKING,{'payment_status': 'Success'}
2,SEARCH,{'search_keywords': 'Dress Kondangan'}
3,ADD_TO_CART,"{'quantity': 2, 'item_price': 249443, 'product_id': 6133}"
4,ADD_TO_CART,"{'quantity': 1, 'item_price': 134504, 'product_id': 6721}"


C:\Users\Miguel\AppData\Local\Temp\ipykernel_12832\1161598943.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_by_event = df_sample.groupby("event_name").apply(lambda x: x.head(3))[["event_metadata"]]


,event_metadata
0,"{'promo_code': 'AZ2022', 'promo_amount': 7047}"
1,"{'promo_code': 'BUYMORE', 'promo_amount': 4903}"
2,"{'promo_code': 'AZ2022', 'promo_amount': 6307}"
3,"{'quantity': 4, 'item_price': 313529, 'product_id': 15315}"
4,"{'quantity': 2, 'item_price': 249443, 'product_id': 6133}"
5,"{'quantity': 1, 'item_price': 134504, 'product_id': 6721}"
6,{'payment_status': 'Success'}
7,{'payment_status': 'Success'}
8,{'payment_status': 'Success'}
9,{'search_keywords': 'Dress Kondangan'}


### Definición de Tablas Auxiliares Derivadas de `event_metadata`

A partir del análisis exploratorio del campo `event_metadata`, se identificaron varios patrones de estructura asociados a distintos tipos de eventos. Para mejorar la accesibilidad y explotabilidad de estos datos, se propone su descomposición en tablas auxiliares específicas.

#### Estructuras detectadas:
1. **Promociones**: Incluyen campos como `promo_code` y `promo_amount`.
2. **Interacción con productos**: Información sobre `product_id`, `item_price`, y `quantity`.
3. **Estado de pago**: Registra el resultado de una operación (`Success`, `Failed`).
4. **Búsquedas**: Contienen el campo `search_keywords`.

#### Objetivo:
- Normalizar la información contenida en `event_metadata`.
- Facilitar la consulta directa por tipo de información.
- Preservar la trazabilidad de los eventos mediante la clave `event_id`.

> Estas tablas actuarán como extensiones de `Click_Stream`, permitiendo un análisis más fino del comportamiento del usuario a lo largo de su navegación.


### Creación de la Tabla Auxiliar `Product_Event_Metadata`

Se ha creado la tabla auxiliar `Product_Event_Metadata` con el objetivo de descomponer y normalizar los detalles de producto presentes en el campo `event_metadata` de la tabla `Click_Stream`.

#### Estructura de la tabla:
- `event_id`: Clave primaria y clave foránea hacia `Click_Stream`.
- `product_id`: Clave foránea hacia la tabla `Products`.
- `quantity`: Cantidad del producto asociada al evento.
- `item_price`: Precio del producto registrado en ese evento.

#### Relaciones:
- Con `ClickStream_Transactions` mediante `event_id`.
- Con `Products` mediante `product_id`.

> Esta tabla permite analizar qué productos son más clicados o visualizados, con qué frecuencia, y qué tipo de interacciones preceden una conversión.


In [5]:
%%time
# SQL para crear y poblar la tabla auxiliar
sql_script = """
CREATE TABLE IF NOT EXISTS Product_Event_Metadata (
    event_id UUID PRIMARY KEY REFERENCES Click_Stream(event_id),
    product_id INT REFERENCES Products(id),
    quantity INT,
    item_price FLOAT
);

INSERT INTO Product_Event_Metadata (event_id, product_id, quantity, item_price)
SELECT 
    event_id,
    (event_metadata->>'product_id')::INT,
    (event_metadata->>'quantity')::INT,
    (event_metadata->>'item_price')::FLOAT
FROM Click_Stream
WHERE event_metadata ? 'product_id';
"""

# Ejecutar el script con gestión segura de conexión
try:
    with engine.begin() as conn:
        conn.execute(text(sql_script))
        print("✅ Tabla 'Product_Event_Metadata' creada y poblada con éxito.")
except Exception as e:
    print("❌ Error al ejecutar el script:", e)

✅ Tabla 'Product_Event_Metadata' creada y poblada con éxito.
CPU times: total: 0 ns
Wall time: 1min 24s


### Creación y Población de la Tabla Auxiliar `Promo_Event_Metadata`

Se ha creado y poblado la tabla auxiliar `Promo_Event_Metadata`, diseñada para almacenar los códigos promocionales y descuentos asociados a eventos de navegación.

#### Estructura de la tabla:
- `event_id`: Clave primaria y clave foránea desde `Click_Stream`.
- `promo_code`: Código promocional aplicado.
- `promo_amount`: Monto de descuento asociado.

> Esta tabla permite analizar el uso de códigos promocionales durante la navegación, su impacto en la conversión y su distribución por canal o tipo de evento.


In [6]:
%%time
# SQL para crear y poblar la tabla Promo_Event_Metadata
sql_script = """
CREATE TABLE IF NOT EXISTS Promo_Event_Metadata (
    event_id UUID PRIMARY KEY REFERENCES Click_Stream(event_id),
    promo_code TEXT,
    promo_amount FLOAT
);

INSERT INTO Promo_Event_Metadata (event_id, promo_code, promo_amount)
SELECT 
    event_id,
    event_metadata->>'promo_code',
    (event_metadata->>'promo_amount')::FLOAT
FROM Click_Stream
WHERE event_metadata ? 'promo_code';
"""

# Ejecutar el script
try:
    with engine.begin() as conn:
        conn.execute(text(sql_script))
        print("✅ Tabla 'Promo_Event_Metadata' creada y poblada con éxito.")
except Exception as e:
    print("❌ Error al ejecutar el script:", e)

✅ Tabla 'Promo_Event_Metadata' creada y poblada con éxito.
CPU times: total: 0 ns
Wall time: 22.3 s


### Creación y Población de la Tabla Auxiliar `Search_Event_Metadata`

Se creó la tabla `Search_Event_Metadata` para almacenar los términos de búsqueda introducidos por los usuarios durante su navegación, extraídos del campo `event_metadata` de la tabla `Click_Stream`.

#### Estructura:
- `event_id`: Clave primaria y clave foránea hacia `Click_Stream`.
- `search_keywords`: Término o frase buscada por el usuario.

> Esta tabla es especialmente útil para analizar la intención del usuario, identificar tendencias de búsqueda y diseñar estrategias de contenido o productos basadas en intereses reales.


In [7]:
%%time
# SQL para crear y poblar la tabla Search_Event_Metadata
sql_script = """
CREATE TABLE IF NOT EXISTS Search_Event_Metadata (
    event_id UUID PRIMARY KEY REFERENCES Click_Stream(event_id),
    search_keywords TEXT
);

INSERT INTO Search_Event_Metadata (event_id, search_keywords)
SELECT 
    event_id,
    event_metadata->>'search_keywords'
FROM Click_Stream
WHERE event_metadata ? 'search_keywords';
"""

# Ejecutar el script
try:
    with engine.begin() as conn:
        conn.execute(text(sql_script))
        print("✅ Tabla 'Search_Event_Metadata' creada y poblada con éxito.")
except Exception as e:
    print("❌ Error al ejecutar el script:", e)

✅ Tabla 'Search_Event_Metadata' creada y poblada con éxito.
CPU times: total: 0 ns
Wall time: 43.7 s


### Creación y Población de la Tabla Auxiliar `Payment_Status_Metadata`

Se ha creado la tabla `Payment_Status_Metadata` para almacenar los resultados de operaciones de pago registradas en los eventos de navegación. Esta información se extrae del campo `event_metadata` de la tabla `Click_Stream`.

#### Estructura:
- `event_id`: Clave primaria y clave foránea hacia `Click_Stream`.
- `payment_status`: Estado de la transacción (ej. "Success", "Failed").

> Esta tabla permite realizar análisis sobre la efectividad del proceso de pago, detectar errores recurrentes y estudiar correlaciones entre comportamiento de navegación y éxito en la transacción.


In [8]:
%%time

# SQL para crear y poblar la tabla Payment_Status_Metadata
sql_script = """
CREATE TABLE IF NOT EXISTS Payment_Status_Metadata (
    event_id UUID PRIMARY KEY REFERENCES Click_Stream(event_id),
    payment_status TEXT
);

INSERT INTO Payment_Status_Metadata (event_id, payment_status)
SELECT 
    event_id,
    event_metadata->>'payment_status'
FROM Click_Stream
WHERE event_metadata ? 'payment_status';
"""

# Ejecutar el script
try:
    with engine.begin() as conn:
        conn.execute(text(sql_script))
        print("✅ Tabla 'Payment_Status_Metadata' creada y poblada con éxito.")
except Exception as e:
    print("❌ Error al ejecutar el script:", e)

✅ Tabla 'Payment_Status_Metadata' creada y poblada con éxito.
CPU times: total: 0 ns
Wall time: 37.6 s


## Resumen Final: Descomposición del Campo `event_metadata`

Como parte del proceso de enriquecimiento y normalización de los datos de comportamiento de usuario, se ha llevado a cabo una descomposición del campo `event_metadata` de la tabla `Click_Stream_transaction`. Este campo contenía información adicional en formato `JSONB`, cuya estructura variaba según el tipo de evento (`event_name`).

### Objetivo:
Transformar datos semi-estructurados en tablas relacionales para:
- Mejorar la accesibilidad y trazabilidad de los datos,
- Facilitar análisis segmentados por tipo de evento o acción,
- Habilitar modelos avanzados de análisis del comportamiento del usuario.

---

### Tablas auxiliares creadas

| Tabla                      | Claves extraídas de `event_metadata`                     | Claves externas |
|---------------------------|----------------------------------------------------------|-----------------|
| `Product_Event_Metadata`  | `product_id`, `quantity`, `item_price`                  | `event_id`, `product_id` |
| `Promo_Event_Metadata`    | `promo_code`, `promo_amount`                            | `event_id`      |
| `Search_Event_Metadata`   | `search_keywords`                                       | `event_id`      |
| `Payment_Status_Metadata` | `payment_status`                                        | `event_id`      |

---

### Proceso aplicado:
1. **Identificación** de claves relevantes en muestras del campo `event_metadata`.
2. **Creación de tablas** auxiliares normalizadas, todas relacionadas con `ClickStream_Transactions` mediante `event_id`.

---

### Resultados:
- Se obtuvo una estructura de datos más rica y relacionalmente sólida.
- Se habilita la consulta directa por atributos específicos del comportamiento de usuario.
- Se sientan las bases para análisis como funnels de conversión, rendimiento de campañas promocionales, eficiencia del pago, y tendencias de búsqueda.

> Este enfoque permite integrar flexibilidad semántica con estructura relacional, facilitando tanto el análisis exploratorio como la modelización avanzada del comportamiento del consumidor.
---

# Estudio Estructural del Modelo de Datos

Con el objetivo de realizar un análisis exhaustivo del comportamiento del consumidor y construir modelos avanzados de segmentación y recomendación, se ha diseñado una base de datos relacional enriquecida, compuesta por tablas principales y auxiliares. A continuación, se presenta un estudio detallado de la estructura actual del modelo de datos.

---

## Tablas Principales

### `Products`
Contiene la información del catálogo de productos disponible en el ecommerce.

- **Campos:** `id`, `gender`, `masterCategory`, `subCategory`, `articleType`, `baseColour`, `season`, `year`, `usage`, `productDisplayName`
- **Relaciones:**
  - Se vincula con `Products_Transactions` y `Product_Event_Metadata` mediante `product_id`.

---

### `Customers`
Almacena los datos de los usuarios registrados en la plataforma.

- **Campos:** `customer_id`, `first_name`, `last_name`, `email`, `gender`, `birthdate`, etc.
- **Relaciones:**
  - Se vincula con `Transactions` mediante `customer_id`.

---

### `Transactions`
Registra las sesiones de compra realizadas por los usuarios.

- **Campos:** `session_id`, `created_at`, `customer_id`, `booking_id`, `product_metadata` (JSONB)
- **Relaciones:**
  - Se vincula con `Customers` mediante `customer_id`.
  - Se descompone en `Products_Transactions` mediante el campo `product_metadata`.

---

### `Click_Stream`
Contiene el log de eventos de navegación (clicks, búsquedas, promociones, etc.).

- **Campos:** `event_id`, `session_id`, `event_name`, `event_time`, `traffic_source`, `event_metadata` (JSONB)
- **Relaciones:**
  - Se vincula con todas las tablas auxiliares de eventos mediante `event_id`.

---

## Tablas Auxiliares

### `Products_Transactions`
Representa la relación muchos-a-muchos entre productos y transacciones.

- **Campos:** `session_id`, `product_id`, `quantity`, `item_price`
- **Origen:** Desanidado desde `Transactions.product_metadata`
- **Relaciones:** une `Products` con `Transactions`.

---

### `Product_Event_Metadata`
Descompone eventos de producto (vista, clic, carrito) desde el campo `event_metadata`.

- **Campos:** `event_id`, `product_id`, `quantity`, `item_price`
- **Origen:** `Click_Stream.event_metadata`
- **Relaciones:** une `Products` con `Click_Stream`.

---

### `Promo_Event_Metadata`
Registra promociones aplicadas durante eventos de navegación o compra.

- **Campos:** `event_id`, `promo_code`, `promo_amount`
- **Origen:** `Click_Stream.event_metadata`

---

### `Search_Event_Metadata`
Contiene los términos de búsqueda introducidos por el usuario.

- **Campos:** `event_id`, `search_keywords`
- **Origen:** `Click_Stream.event_metadata`

---

### `Payment_Status_Metadata`
Almacena el estado del pago asociado a ciertos eventos.

- **Campos:** `event_id`, `payment_status`
- **Origen:** `Click_Stream.event_metadata`

---

## Relaciones entre Tablas (Resumen)

```text
Customers ─────┐
               └──▶ Transactions ───▶ Products_Transactions ◀── Products
Click_Stream ──┬──▶ Product_Event_Metadata ◀── Products
               ├──▶ Promo_Event_Metadata
               ├──▶ Search_Event_Metadata
               └──▶ Payment_Status_Metadata
```
### **Conclusión**
Esta arquitectura relacional permite un análisis integral del comportamiento del consumidor, facilitando:

- Estudios de patrones de compra y navegación.

- Segmentación basada en RFM, actividad o intereses.

- Recomendación de productos personalizada.

- Medición de efectividad de campañas y promociones.

## Enriquecimiento de la Tabla `Product_Event_Metadata` con Indicador de Compra

### Objetivo

El propósito de esta operación es **identificar si un producto visualizado durante una sesión fue finalmente comprado**, permitiendo así diferenciar entre simples interacciones y conversiones reales. Esta información es clave para calcular:

- La tasa de conversión por producto o categoría.
- El abandono de productos tras visualización o añadido al carrito.
- Métricas de eficacia de recomendaciones o promociones.

### Justificación

La tabla `Product_Event_Metadata` captura interacciones con productos (vistas, clics, etc.), pero no distingue si dichas interacciones terminaron en compra. En cambio, la tabla `Products_Transactions` contiene los productos comprados efectivamente por sesión.

Al cruzar ambos conjuntos de datos (`session_id`, `product_id`), se puede enriquecer `Product_Event_Metadata` con un campo booleano `was_purchased`, que indica si ese producto fue adquirido durante la misma sesión donde ocurrió el evento.

Este nuevo campo habilita análisis más profundos de comportamiento y permitirá construir mejores modelos de recomendación, segmentación o embudo de conversión.

### Proceso

1. Añadir la columna `was_purchased` a la tabla `Product_Event_Metadata`.
2. Actualizar su valor a `TRUE` si existe una coincidencia con la tabla `Products_Transactions`.
3. Tener en cuenta todos los campos, el mismo producto puede estar en la tabla `Product_Event_Metadata` pero con otro precio.


In [9]:
%%time

# Script SQL
sql_script = """
-- Añadir columna si no existe
DO $$
BEGIN
    IF NOT EXISTS (
        SELECT 1
        FROM information_schema.columns
        WHERE table_name='product_event_metadata' AND column_name='was_purchased'
    ) THEN
        ALTER TABLE Product_Event_Metadata ADD COLUMN was_purchased BOOLEAN DEFAULT FALSE;
    END IF;
END$$;

-- Actualizar valores a TRUE si el producto fue comprado en la misma sesión
UPDATE Product_Event_Metadata
SET was_purchased = TRUE
FROM Click_Stream cs
JOIN Products_Transactions pt
  ON cs.session_id = pt.session_id
WHERE Product_Event_Metadata.event_id = cs.event_id
  AND Product_Event_Metadata.product_id = pt.product_id
  AND Product_Event_Metadata.quantity = pt.quantity
  AND Product_Event_Metadata.item_price = pt.item_price;
"""

# Ejecutar el script
with engine.begin() as conn:
    conn.execute(text(sql_script))
    print("✅ Campo 'was_purchased' añadido y actualizado con éxito en Product_Event_Metadata.")


✅ Campo 'was_purchased' añadido y actualizado con éxito en Product_Event_Metadata.
CPU times: total: 0 ns
Wall time: 53.6 s


# Enriquecimiento de la Tabla `Products` con Imágenes

Para mejorar la calidad de los datos y enriquecer el análisis visual en futuras etapas del proyecto, se ha decidido incorporar imágenes representativas de cada producto en la tabla `Products`.

---

## Objetivo:
- Realizar un scraping de imágenes desde motores de búsqueda (**Bing**, **Google** y **Yandex**) utilizando el nombre del producto (`productDisplayName`) como query.
- Almacenar la URL de la imagen más relevante directamente en la tabla, en un campo adicional llamado `image_url`.

---

## Proceso:
### **Ampliación de la Tabla:**
   - Se añade un campo `image_url` en la tabla `Products` para almacenar la referencia de la imagen.

### **Scraping de Imágenes:**
   - Usando técnicas de scraping, se buscan imágenes relacionadas con el producto:
     - **Bing**, **Google** y **Yandex** se utilizan como fuentes.
     - En caso de no encontrar una imagen, se asigna una imagen genérica por defecto.

### **Actualización de la Tabla:**
   - Las URLs obtenidas se insertan en el campo `image_url` para su futura visualización y análisis.


In [3]:
%%time
# Script SQL para añadir el campo image_url
sql_script = """
ALTER TABLE Products ADD COLUMN IF NOT EXISTS image_url TEXT;
"""

# Ejecutar el script
try:
    with engine.begin() as conn:
        conn.execute(text(sql_script))
    print("✅ Columna 'image_url' añadida exitosamente en la tabla 'Products'.")
except Exception as e:
    print(f"❌ Error al añadir la columna 'image_url': {e}")

✅ Columna 'image_url' añadida exitosamente en la tabla 'Products'.
CPU times: total: 0 ns
Wall time: 203 ms


# Scraping de Imágenes para Enriquecer la Tabla `Products`

Para enriquecer visualmente los datos de nuestra tabla `Products`, hemos implementado un proceso de **scraping de imágenes** desde los motores de búsqueda más populares: **Bing**, **Google** y **Yandex**.

---

## **Objetivo:**
- Obtener imágenes representativas para cada producto de la tabla `Products`.
- Poblar el campo `image_url` con la URL de la imagen más relevante encontrada.

---

## **Motores de Búsqueda Utilizados:**
1. **Bing Images:**  
   - Se realiza una búsqueda directa a través de peticiones HTTP y se parsea el HTML para extraer la URL de la imagen más relevante.

2. **Google Images:**  
   - Usamos Selenium en modo *headless* para interactuar con la búsqueda de imágenes y extraer la primera URL disponible.

3. **Yandex Images:**  
   - Se ejecuta una búsqueda adicional en Yandex para maximizar las posibilidades de encontrar una imagen si Bing y Google fallan.

---

## **Proceso de Búsqueda:**
1. Se intenta primero en **Bing**.
2. Si no se encuentra una imagen, se pasa a **Google**.
3. Si Google tampoco devuelve una URL válida, se realiza una búsqueda en **Yandex**.
4. En caso de que los tres motores de búsqueda fallen, se asigna una **imagen genérica** por defecto.

---

## **Próximo Paso:**
El siguiente paso será recorrer los productos almacenados en la base de datos, ejecutar el scraping para cada uno de ellos y actualizar el valor en el campo `image_url` dentro de la tabla `Products`.


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random
import csv
import json
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# ==== Configuración general ====
GENERIC_IMAGE_URL = "https://upload.wikimedia.org/wikipedia/commons/1/14/No_Image_Available.jpg"  # <-- Pon tu imagen genérica aquí
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}

# ==== Configurar Selenium ====
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--window-size=1920,1080")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# ==== Funciones de búsqueda ====
def search_image_bing(query):
    try:
        query = query.replace(' ', '+')
        url = f"https://www.bing.com/images/search?q={query}&form=HDRSC2"
        res = requests.get(url, headers=HEADERS, timeout=10)
        if res.status_code != 200:
            return None
        soup = BeautifulSoup(res.text, 'html.parser')
        image_elements = soup.find_all('a', class_='iusc')
        if not image_elements:
            return None
        first_image_json = image_elements[0].get('m')
        image_metadata = json.loads(first_image_json)
        return image_metadata.get('murl')
    except:
        return None


def search_image_google(query):
    try:
        driver.get(f"https://www.google.com/search?q={query}&tbm=isch")
        time.sleep(2)
        images = driver.find_elements(By.TAG_NAME, "img")
        for img in images:
            src = img.get_attribute("data-src") or img.get_attribute("src")
            if src and src.startswith('https://encrypted-tbn0.gstatic.com/'):
                return src
        return None
    except:
        return None

def search_image_yandex(query):
    try:
        driver.get(f"https://yandex.com/images/search?text={query}")
        time.sleep(2)
        images = driver.find_elements(By.TAG_NAME, "img")
        for img in images:
            src = img.get_attribute("src")
            if src and src.startswith('https://') and 'nimages' not in src:
                return src
        return None
    except:
        return None

def find_best_image(query):
    print(f"Searching Bing for: {query}")
    img = search_image_bing(query)
    if img:
        print("✅ Found in Bing")
        return img

    print(f"Not found in Bing. Trying Google...")
    img = search_image_google(query)
    if img:
        print("✅ Found in Google")
        return img

    print(f"Not found in Google. Trying Yandex...")
    img = search_image_yandex(query)
    if img:
        print("✅ Found in Yandex")
        return img

    print("❌ No image found. Assigning generic image.")
    return GENERIC_IMAGE_URL

# ==== Parámetros de trabajo ====
# ==== Consulta SQL para extraer productos ====
query = "SELECT id, productdisplayname FROM Products"

output_csv = 'final_product_images.csv'

# ==== Cargar productos ====
df = pd.read_sql(query, engine)
df = df.rename(columns={'productdisplayname': 'productDisplayName'})
products = df[['id', 'productDisplayName']].dropna()

# ==== Cargar progreso previo si existe ====
if os.path.exists(output_csv):
    done = pd.read_csv(output_csv)
    done_ids = set(done['id'].astype(str))
    print(f"Resuming. Already processed {len(done_ids)} products.")
else:
    done_ids = set()
    with open(output_csv, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["id", "ProductDisplayName", "ImageURL"])
    print("Starting new file.")

# ==== Procesar productos ====
with open(output_csv, mode='a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    processed_count = 0

    for idx, row in enumerate(products.itertuples(index=False), 1):
        product_id = str(row.id)
        desc = row.productDisplayName

        if product_id in done_ids:
            continue  # Ya procesado

        try:
            print(f"\n[{idx}] Searching for: {desc}")
            image_url = find_best_image(desc)
            writer.writerow([product_id, desc, image_url])
            f.flush()
            
            processed_count += 1

            # Simular comportamiento humano
            sleep_time = random.uniform(2, 5)
            print(f"⏳ Waiting {sleep_time:.2f} seconds before next search...")
            time.sleep(sleep_time)

            # Cada 20-30 productos, hacer una pausa larga
            if processed_count % random.randint(20, 30) == 0:
                long_sleep = random.uniform(60, 90)
                print(f"💤 Taking a long break of {long_sleep:.2f} seconds...")
                time.sleep(long_sleep)

        except Exception as e:
            print(f"Error processing {desc}: {e}")
            writer.writerow([product_id, desc, GENERIC_IMAGE_URL])
            f.flush()
            continue

# ==== Cerrar Selenium ====
driver.quit()

print(f"\n✅ ¡Todos los productos procesados! Resultado guardado en {output_csv}")

# Justificación del uso de un CSV para almacenar los resultados del Scraping

El proceso de scraping de imágenes para los productos implica realizar múltiples peticiones HTTP a motores de búsqueda (Bing, Google, Yandex). Esto puede implicar tiempos de espera elevados, errores intermitentes en las conexiones y bloqueos temporales por parte de los servidores. Por este motivo, la decisión de almacenar los resultados en un archivo CSV se justifica por las siguientes razones:

---

### **1. Resiliencia ante errores de red**
El scraping es un proceso propenso a fallos por:
- Errores de conexión (timeouts, denegaciones del servidor, etc.).
- Cambios en la estructura HTML de las páginas objetivo.
- Bloqueos temporales por parte de los servidores al detectar múltiples peticiones en poco tiempo.

Al guardar los resultados en un CSV de manera incremental, es posible **pausar y reanudar el proceso** en caso de fallos sin perder el progreso, ya que solo se reintentan los productos que no están en el archivo.

---

### **2. Evitar transacciones masivas en la base de datos**
Realizar múltiples inserciones en la base de datos mientras se ejecuta el scraping podría:
- Aumentar la latencia en la base de datos.
- Generar locks (bloqueos) en las tablas afectadas.
- Causar inconsistencias en caso de una caída inesperada del proceso.

El CSV permite un enfoque **batch** (por lotes) para la inserción de datos. Una vez finalizado el scraping, se puede procesar el archivo y hacer un único `BULK INSERT`, optimizando el rendimiento y reduciendo el riesgo de errores transaccionales.

---

### **3. Facilidad de revisión y validación**
El CSV es un formato **fácilmente auditable**:
- Podemos revisar manualmente las URLs obtenidas.
- Es sencillo identificar imágenes faltantes o URLs erróneas.
- En caso de problemas, se puede modificar el CSV antes de subir los datos finales a la base de datos.

---

### **4. Integración con procesos ETL**
Este enfoque facilita la integración en un flujo ETL:
1. **Extract:** Scraping de las imágenes.
2. **Transform:** Validación y limpieza de URLs en el CSV.
3. **Load:** Carga optimizada en la base de datos.

---

### **Flujo propuesto:**
1️ Scraping → 2️ Almacenamiento en CSV → 3️ Validación → 4️ Carga en la base de datos


In [26]:
%%time
# ==== Leer el CSV generado ====
csv_path = 'final_product_images.csv'
df = pd.read_csv(csv_path)
df = df.rename(columns={'ImageURL': 'image_url'})

# ==== Actualización en bloques con COMMIT explícito ====
update_query = text("""
    UPDATE Products
    SET image_url = :image_url
    WHERE id = :id
""")

# Dividir el DataFrame en bloques de 1000
def chunk_df(dataframe, chunk_size=1000):
    """Divide un DataFrame en bloques más pequeños"""
    for i in range(0, len(dataframe), chunk_size):
        yield dataframe.iloc[i:i + chunk_size]

# Conexión directa para múltiples actualizaciones
with engine.connect() as connection:
    for chunk in chunk_df(df):
        trans = connection.begin()  # Iniciar transacción
        try:
            for _, row in chunk.iterrows():
                connection.execute(update_query, {
                    "image_url": row['image_url'],
                    "id": int(row['id'])
                })
            trans.commit()  # Realizar el commit explícito
        
        except Exception as e:
            trans.rollback()  # Revertir si hay un error
            print(f"❌ Error en la transacción: {e}")
                
print("\n✅ Las URLs de las imágenes se han actualizado correctamente en la tabla Products.")



✅ Las URLs de las imágenes se han actualizado correctamente en la tabla Products.
CPU times: total: 5.83 s
Wall time: 8.87 s


## **Conclusión**
El scraping de imágenes permitió enriquecer la base de datos de productos con URLs de imágenes de alta calidad, optimizando la presentación visual del catálogo. Gracias al uso del CSV, el proceso se mantuvo resiliente y fácilmente auditable, facilitando una actualización limpia y efectiva en PostgreSQL.